In [1]:
import scipy.io as sio
import numpy as np
import os
import tensorboard
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt

from torch.utils.tensorboard import SummaryWriter
from utils.model import DownSampleConv, UpSampleConv

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


In [2]:
D = sio.loadmat('/home/hou63/pj2/Nematic_RL/datas/D.mat')['D']
# print keys
print(D.shape)
d11 = D[:,:,0,40:]
d12 = D[:,:,1,40:]
print(d11.shape)
# put axis 2 to 0
d11 = np.moveaxis(d11, 2, 0)
d12 = np.moveaxis(d12, 2, 0)
print(d11.shape)
ds = np.stack((d11, d12), axis=1)
print(ds.shape)
ds = torch.tensor(ds, dtype=torch.float32, device=device)

(256, 256, 3, 257)
(256, 256, 217)
(217, 256, 256)
(217, 2, 256, 256)


In [ ]:
# plot D
plt.figure()
plt.imshow(d11[:,:,0])
plt.colorbar()
plt.show()

In [3]:

encoder = DownSampleConv()
decoder = UpSampleConv()

encoder = encoder.to(device)
decoder = decoder.to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(encoder.parameters(), lr=0.001)

# 训练参数
num_epochs = 10
dataset = TensorDataset(ds)  # 将 ds 封装为 dataset
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

NameError: name 'ds' is not defined

In [4]:
num_epochs = 200
save_path = '/home/hou63/pj2/Nematic_RL/log_model/encoder_checkpoint.pth'  # 模型保存路径
best_loss = float('inf')  # 初始最优损失值设为无穷大

# 开始训练
for epoch in range(num_epochs):
    encoder.train()  # 设置 encoder 为训练模式
    decoder.train()  # 设置 decoder 为训练模式
    
    total_loss = 0.0

    for data in dataloader:
        # 将数据移到设备上
        x = data[0].to(device)
        
        # 前向传播: encoder 和 decoder
        encoded = encoder(x)
        reconstructed = decoder(encoded)
        
        # 计算损失
        loss = criterion(reconstructed, x)  # 自监督损失：重建误差
        
        # 反向传播与优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    avg_loss = total_loss / len(dataloader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')
    
    # 保存模型：当当前损失小于最佳损失时，保存模型
    if avg_loss < best_loss:
        best_loss = avg_loss
        torch.save(encoder.state_dict(), save_path)
        print(f'Model saved with loss {best_loss:.4f}')

NameError: name 'dataloader' is not defined

In [6]:
# load encoder model
encoder.load_state_dict(torch.load(save_path))
encoder.eval()



/tmp/ipykernel_562286/2469840783.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  encoder.load_state_dict(torch.load(save_path))


DownSampleConv(
  (conv1): Conv2d(2, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (norm1): BatchNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (norm2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)